# This file shows the process of creating an EA using ARIEL 

In [2]:
# Standard library
import random
from typing import Literal, cast

# Pretty little errors and progress bars
from rich.console import Console
from rich.traceback import install

# Third-party libraries
import numpy as np

# Local libraries
from ariel.ec.a000 import IntegerMutator
from ariel.ec.a001 import Individual
from ariel.ec.a005 import Crossover
from ariel.ec.a004 import EASettings, EAStep, EA, Population

# Function to show fitness landscape
from plot_fit_per_gen import plot_fit_per_gen


In [3]:
def visualize_solution(solution):
    """Visualize the placement of queens on the chessboard."""


    n = len(solution)
    for i in range(n):
        rep = ""
        for j in range(n):
            rep += "Q " if j == solution[i] else ". " 

        print(rep)

example_solution = [0,1,2,3]

visualize_solution(example_solution)

Q . . . 
. Q . . 
. . Q . 
. . . Q 


In [ ]:
#### Define the fitness function
def evaluate_solution_n_queens(solution):
    """Calculate the fitness of an solution."""


    attacks = 0
    n = len(solution)
    for i in range(n):
        for j in range(i + 1, n):
            if solution[i] == solution[j] or abs(solution[i] - solution[j]) == abs(i - j):
                attacks += 1
                break  # Break out of the loop once an attack is found for the current queen
    fitness = n - attacks

    return float(fitness)

def evaluate_ind(ind: Individual) -> float:
    """Evaluate an individual by calculating its fitness using the Ackley function."""

    return evaluate_solution_n_queens(cast("list[float]", ind.genotype))

def evaluate_pop(population: Population) -> Population:
    """Evaluate a population by calculating the fitness of each individual."""
    for ind in population:
        if ind.requires_eval:
            ind.fitness = evaluate_ind(ind)
    return population



In [5]:
# A seed is optional, but it helps with reproducibility
SEED = None  # e.g., 42

# The database has a few handling modes
    # "delete" will delete the existing database
    # "halt" will stop the execution if a database already exists
DB_HANDLING_MODES = Literal["delete", "halt"]

# Initialize RNG
RNG = np.random.default_rng(SEED)

# Initialize rich console and traceback handler
install()
console = Console()

In [6]:
# Set config
config = EASettings()
config.is_maximisation = False
config.db_handling = "delete"
config.target_population_size = 100


In [10]:
def create_individual(num_dims) -> Individual:
    ind = Individual()
    ind.genotype = np.random.permutation(num_dims - 1).tolist()
    return ind

def parent_selection(population: Population) -> Population:
    """Tournament Selection"""

    tournament_size: int = 3

    # Ensure all individuals have a tags dict and reset parent-selection tag
    for ind in population:
        if ind.tags is None:
            ind.tags = {}
        ind.tags['ps'] = False

    # Decide how many parents we want (even number)
    num_parents = (len(population) // 2) * 2
    if num_parents == 0 and len(population) >= 2:
        num_parents = 2

    winners = []
    for _ in range(num_parents):
        # sample competitors with replacement
        competitors = [random.choice(population) for _ in range(tournament_size)]

        # pick best competitor depending on maximisation/minimisation
        if config.is_maximisation:
            winner = max(competitors, key=lambda ind: ind.fitness)
        else:
            winner = min(competitors, key=lambda ind: ind.fitness)

        winners.append(winner)

    # mark winners as parents
    for w in winners:
        w.tags['ps'] = True

    return population
    
def crossover(population: Population) -> Population:
    """One point crossover"""

    parents = [ind for ind in population if ind.tags.get("ps", False)]
    for idx in range(0, len(parents), 2):
        parent_i = parents[idx]
        parent_j = parents[idx]
        genotype_i, genotype_j = Crossover.one_point(
            cast("list[float]", parent_i.genotype),
            cast("list[float]", parent_j.genotype),
        )

        # First child
        child_i = Individual()
        child_i.genotype = genotype_i
        child_i.tags = {"mut": True}
        child_i.requires_eval = True

        # Second child
        child_j = Individual()
        child_j.genotype = genotype_j
        child_j.tags = {"mut": True}
        child_j.requires_eval = True

        population.extend([child_i, child_j])
    return population

def mutation(population: Population) -> Population:
    for ind in population:
        if ind.tags.get("mut", False):
            genes = cast("list[int]", ind.genotype)
            mutated = IntegerMutator.integer_creep(
                individual=genes,
                span=1,
                mutation_probability=0.5,
            )
            ind.genotype = mutated
            ind.requires_eval = True
    return population

def survivor_selection(population: Population) -> Population:

    # Shuffle population to avoid bias
    random.shuffle(population)
    current_pop_size = len(population)

    # for idx in range(len(population)):
    for idx in range(0, len(population) - 1, 2):
        ind_i = population[idx]
        ind_j = population[idx + 1]

        # Kill worse individual
        if ind_i.fitness > ind_j.fitness and config.is_maximisation:
            ind_j.alive = False
        else:
            ind_i.alive = False

        # Termination condition
        current_pop_size -= 1
        if current_pop_size <= config.target_population_size:
            break
    return population

In [11]:
def main(pop_size) -> EA:
    """Entry point."""
    # Create initial population
    population_list = [create_individual(num_dims=16) for _ in range(pop_size)]
    population_list = evaluate_pop(population_list)

    # Create EA steps
    ops = [
        EAStep("parent_selection", parent_selection),
        EAStep("crossover", crossover),
        EAStep("mutation", mutation),
        EAStep("evaluation", evaluate_pop),
        EAStep("survivor_selection", survivor_selection),
    ]

    # Initialize EA
    ea = EA(
        population_list,
        operations=ops,
        num_of_generations=100,
    )

    ea.run()

    best = ea.get_solution("best", only_alive=False)
    console.log(f"Best fitness: {best.fitness}")
    visualize_solution(best.genotype)

    median = ea.get_solution("median", only_alive=False)
    console.log(f"Median fitness: {median.fitness}")
    visualize_solution(median.genotype)

    worst = ea.get_solution("worst", only_alive=False)
    console.log(f"Worst fitness: {median.fitness}")
    visualize_solution(worst.genotype)


    return ea

In [12]:
ea = main(pop_size=100)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 ea = main(pop_size=100)                                                                      │
│   2                                                                                              │
│                                                                                                  │
│ in main:5                                                                                        │
│                                                                                                  │
│    2 │   """Entry point."""                                                                      │
│    3 │   # Create initial population                                                             │
│    4 │   population_list = [create_individual(num_dims=16) for _ in range(pop_size)]             │
│ ❱  5 │   population_list = evaluate_pop(population_list)                                         │
│    6 │                                                                                           │
│    7 │   # Create EA steps                                                                       │
│    8 │   ops = [                                                                                 │
│                                                                                                  │
│ in evaluate_pop:26                                                                               │
│                                                                                                  │
│   23 │   """Evaluate a population by calculating the fitness of each individual."""              │
│   24 │   for ind in population:                                                                  │
│   25 │   │   if ind.requires_eval:                                                               │
│ ❱ 26 │   │   │   ind.fitness = evaluate_ind(ind)                                                 │
│   27 │   return population                                                                       │
│   28                                                                                             │
│   29                                                                                             │
│                                                                                                  │
│ d:\University\EC TA\ariel\.venv\Lib\site-packages\sqlmodel\main.py:830 in __setattr__            │
│                                                                                                  │
│    827 │   │   │   # Set in Pydantic model to trigger possible validation changes, only for      │
│    828 │   │   │   # non relationship values                                                     │
│    829 │   │   │   if name not in self.__sqlmodel_relationships__:                               │
│ ❱  830 │   │   │   │   super().__setattr__(name, value)                                          │
│    831 │                                                                                         │
│    832 │   def __repr_args__(self) -> Sequence[Tuple[Optional[str], Any]]:                       │
│    833 │   │   # Don't show SQLAlchemy private attributes                                        │
│                                                                                                  │
│ d:\University\EC TA\ariel\.venv\Lib\site-packages\pydantic\main.py:1033 in __setattr__           │
│                                                                                                  │
│   1030 │   │   │   │   setattr_handler(self, name, value)                                        │
│   1031 │   │   │   # if None is returned from _setattr_hand

In [ ]:
plot_fit_per_gen()